In [1]:
import os
%pwd

'/home/towet/Desktop/OpenProjects/Malaria-Cell-Histopathology-Classification-Transfer-Learning/research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/towet/Desktop/OpenProjects/Malaria-Cell-Histopathology-Classification-Transfer-Learning'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    dataset: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
from MalariaCellHistopathologyClassification.constants import *
from MalariaCellHistopathologyClassification.utils.common import read_yaml, create_directories, save_json

2024-03-05 15:04:02.762215: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-05 15:04:02.818843: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 15:04:02.818917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 15:04:02.821276: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 15:04:02.834968: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-05 15:04:04.708734: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [11]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        
        dataset = os.path.join("artifacts", "data_ingestion", "dataset/")
        eval_config = EvaluationConfig(
            model_path="artifacts/training/Malaria_cell_TF.h5",
            mlflow_uri="https://dagshub.com/Towet-Tum/Malaria-Cell-Histopathology-Classification-Transfer-Learning.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMG_SHAPE,
            params_batch_size=self.params.BATCH_SIZE,
            dataset=str(dataset)
        )
        return eval_config

In [7]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from MalariaCellHistopathologyClassification.utils.common import load__process_dataset

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config



    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        _,_,test_ds, class_names = load__process_dataset(self.config.dataset)
        self.model = self.load_model(self.config.model_path)
        self.score = self.model.evaluate(test_ds)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG19-Malaria-Cell")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-03-05 15:06:46,421: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-05 15:06:46,428: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-05 15:06:46,429: INFO: common: created directory at: artifacts]
Found 27558 files belonging to 2 classes.
Using 22047 files for training.
Found 27558 files belonging to 2 classes.
Using 5511 files for validation.


2024-03-05 15:07:05.102650: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-03-05 15:07:07.442916: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-03-05 15:07:11.368152: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-03-05 15:07:11.453928: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-03-05 15:07:12.776379: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


34/34 [==============================] - 1548s 46s/step - loss: 0.1314 - accuracy: 0.9605
[2024-03-05 15:32:45,665: INFO: common: json file saved at: scores.json]


2024/03/05 15:32:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-03-05 15:32:51,598: INFO: builder_impl: Assets written to: /tmp/tmplze98qkq/model/data/model/assets]
